In [11]:
!pip install transformers datasets torch nltk rouge-score pandas tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1


In [19]:
import pandas as pd
from datasets import Dataset
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
import torch
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import numpy as np
import warnings
from IPython.display import display, HTML
import os # Importar os para la verificación de rutas

# Configuración de rutas
CSV_PATH = r"C:\DATA\FERNANDOHC\EDUCACION\MAESTRIA\UNI_MAI\SEMESTRE_3\MIA-204ProyectoDeInvestigacion1\Proyecto_Traductor_Esp_Quechua\datos\corpus_trad.csv"
MODEL_PATH_V1 = r"C:\DATA\FERNANDOHC\EDUCACION\MAESTRIA\UNI_MAI\SEMESTRE_3\MIA-204ProyectoDeInvestigacion1\Proyecto_Traductor_Esp_Quechua\modelos\Transformers\mi_modelo.pt"
MODEL_PATH_V2 = r"C:\DATA\FERNANDOHC\EDUCACION\MAESTRIA\UNI_MAI\SEMESTRE_3\MIA-204ProyectoDeInvestigacion1\Proyecto_Traductor_Esp_Quechua\modelos\Transformers\mi_modelo_quechua_v2.pt"

# --- CAMBIO CLAVE AQUÍ ---
# Ya que no tienes la carpeta del tokenizer localmente, lo cargamos por su nombre
# desde Hugging Face Hub. Es VITAL que este nombre corresponda al modelo base
# con el que tus modelos .pt fueron entrenados (ej. "facebook/bart-base").
TOKENIZER_NAME_OR_PATH = "facebook/bart-base" 

# Parámetros de evaluación
NUM_SENTENCES = 50
NUM_RUNS = 5

# Configurar dispositivo para PyTorch (GPU si está disponible, sino CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo de ejecución: {device}")

# Ignorar warnings que puedan surgir de librerías
warnings.filterwarnings('ignore', category=UserWarning)

# --- Verificación de rutas de archivos (útil para depuración) ---
print(f"\n--- Verificación de rutas de archivos ---")
print(f"DEBUG: CSV_PATH = '{CSV_PATH}' - Existe: {os.path.exists(CSV_PATH)}")
print(f"DEBUG: MODEL_PATH_V1 = '{MODEL_PATH_V1}' - Existe: {os.path.exists(MODEL_PATH_V1)}")
print(f"DEBUG: MODEL_PATH_V2 = '{MODEL_PATH_V2}' - Existe: {os.path.exists(MODEL_PATH_V2)}")
print(f"DEBUG: TOKENIZER_NAME_OR_PATH = '{TOKENIZER_NAME_OR_PATH}' (Se cargará desde Hugging Face Hub)")

Dispositivo de ejecución: cpu

--- Verificación de rutas de archivos ---
DEBUG: CSV_PATH = 'C:\DATA\FERNANDOHC\EDUCACION\MAESTRIA\UNI_MAI\SEMESTRE_3\MIA-204ProyectoDeInvestigacion1\Proyecto_Traductor_Esp_Quechua\datos\corpus_trad.csv' - Existe: True
DEBUG: MODEL_PATH_V1 = 'C:\DATA\FERNANDOHC\EDUCACION\MAESTRIA\UNI_MAI\SEMESTRE_3\MIA-204ProyectoDeInvestigacion1\Proyecto_Traductor_Esp_Quechua\modelos\Transformers\mi_modelo.pt' - Existe: True
DEBUG: MODEL_PATH_V2 = 'C:\DATA\FERNANDOHC\EDUCACION\MAESTRIA\UNI_MAI\SEMESTRE_3\MIA-204ProyectoDeInvestigacion1\Proyecto_Traductor_Esp_Quechua\modelos\Transformers\mi_modelo_quechua_v2.pt' - Existe: True
DEBUG: TOKENIZER_NAME_OR_PATH = 'facebook/bart-base' (Se cargará desde Hugging Face Hub)


In [21]:
def load_and_prepare_data(csv_path):
    """
    Carga un archivo CSV, lo procesa para extraer pares de texto
    y lo convierte en un objeto Dataset de Hugging Face.
    """
    try:
        df = pd.read_csv(
            csv_path,
            usecols=['source', 'target'], # Solo nos interesan estas columnas
            encoding='utf-8',
            on_bad_lines='warn' # Advertir sobre líneas mal formadas en lugar de fallar
        )
    except Exception as e:
        print(f"Error al leer el CSV: {e}")
        return Dataset.from_list([]) # Retornar un Dataset vacío si hay un error
    
    data_pairs = []
    # Iterar sobre las filas del DataFrame para crear pares de datos
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Procesando datos"):
        # Asegurarse de que ambas columnas 'source' y 'target' no sean NaN
        if pd.notna(row['source']) and pd.notna(row['target']):
            data_pairs.append({
                "spanish": str(row['source']).strip(), # Asegurar que es string y eliminar espacios
                "quechua": str(row['target']).strip()
            })
    
    print(f"\nTotal de pares válidos cargados: {len(data_pairs)}")
    if data_pairs:
        # Calcular y mostrar la longitud promedio de las oraciones
        avg_len_spanish = sum(len(p['spanish']) for p in data_pairs) / len(data_pairs)
        avg_len_quechua = sum(len(p['quechua']) for p in data_pairs) / len(data_pairs)
        print(f"Longitud promedio español: {avg_len_spanish:.1f} caracteres")
        print(f"Longitud promedio quechua: {avg_len_quechua:.1f} caracteres")
    
    return Dataset.from_list(data_pairs) # Convertir la lista de diccionarios a un objeto Dataset

# Cargar y preparar el dataset al inicio
raw_dataset = load_and_prepare_data(CSV_PATH)

Procesando datos: 100%|██████████| 262855/262855 [00:34<00:00, 7678.73it/s] 



Total de pares válidos cargados: 262855
Longitud promedio español: 117.1 caracteres
Longitud promedio quechua: 120.1 caracteres


In [22]:
def calculate_metrics(predictions, references):
    """
    Calcula las métricas BLEU y ROUGE-L para un conjunto de predicciones y referencias.
    """
    # BLEU Score: Mide la similitud n-grama entre la traducción y la referencia.
    # nltk.translate.bleu_score.corpus_bleu espera:
    # - references: Una lista de referencias, donde cada referencia es una lista de tokens (palabras).
    #   Como solo tenemos una referencia por predicción, la envolvemos en otra lista: [[ref1_tokens], [ref2_tokens]].
    # - predictions: Una lista de traducciones, donde cada traducción es una lista de tokens.
    bleu = corpus_bleu([[ref.split()] for ref in references], 
                       [pred.split() for pred in predictions])
    
    # ROUGE-L Score: Mide la superposición de secuencias más largas comunes (LCS).
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, pred in zip(references, predictions):
        score = scorer.score(ref, pred)['rougeL'].fmeasure # Obtenemos el F-measure de ROUGE-L
        rouge_scores.append(score)
    rouge = np.mean(rouge_scores) # Calculamos la media de todos los scores ROUGE-L
    
    return {"bleu": bleu, "rouge": rouge}

def load_local_model(model_path, tokenizer_name_or_path):
    """
    Carga un modelo de PyTorch guardado localmente (.pt) y su tokenizador
    desde Hugging Face Hub.
    """
    # Cargar tokenizer: AutoTokenizer detectará el tipo de tokenizador por su nombre
    # o desde los archivos si se le pasa una ruta local.
    # Aquí lo cargamos desde el nombre en el Hub (requiere internet la primera vez).
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
    
    # Cargar configuración del modelo: Es esencial que la configuración coincida con
    # la arquitectura del modelo que se entrenó y se guardó en .pt.
    # Usamos el mismo nombre que para el tokenizer si son consistentes (como en BART).
    config = AutoConfig.from_pretrained(tokenizer_name_or_path)
    
    # Crear una instancia del modelo con la configuración cargada
    model = AutoModelForSeq2SeqLM.from_config(config).to(device)
    
    # --- SOLUCIÓN AL UnpicklingError: Añadir weights_only=False ---
    # Esto es necesario si el archivo .pt se guardó con una versión antigua de PyTorch
    # o si contiene metadatos además de solo los pesos.
    # Solo usar si confías en el origen del archivo.
    state_dict = torch.load(model_path, map_location=device, weights_only=False)
    
    # Cargar los pesos (state_dict) en la instancia del modelo
    model.load_state_dict(state_dict)
    
    # Poner el modelo en modo evaluación (desactiva dropout, batch norm, etc.)
    model.eval() 
    
    return tokenizer, model

def evaluate_model(model_path, dataset_subset, tokenizer_name_or_path):
    """
    Evalúa un modelo de traducción en un subconjunto de datos.
    Carga el modelo y el tokenizador, genera predicciones y calcula métricas.
    """
    # Cargar el tokenizador y el modelo usando la función auxiliar
    tokenizer, model = load_local_model(model_path, tokenizer_name_or_path)
    
    predictions = [] # Para almacenar las traducciones generadas por el modelo
    references = []  # Para almacenar las traducciones de referencia (ground truth)

    # Desactivar el cálculo de gradientes durante la inferencia para ahorrar memoria y tiempo
    with torch.no_grad(): 
        for example in tqdm(dataset_subset, desc="Evaluando modelo"):
            # Tokenizar el texto de entrada (español)
            inputs = tokenizer(
                example["spanish"], 
                return_tensors="pt",       # Retornar tensores de PyTorch
                max_length=128,            # Longitud máxima de secuencia de entrada
                truncation=True,           # Truncar si excede max_length
                padding="max_length"       # Rellenar con padding hasta max_length
            ).to(device) # Mover los inputs al dispositivo (GPU/CPU)
            
            # Generar la traducción
            # num_beams=5: Usa beam search con 5 haces para una mejor calidad de generación
            # early_stopping=True: Detiene la generación tan pronto como se encuentran las secuencias completas
            output = model.generate(**inputs, max_length=128, num_beams=5, early_stopping=True)
            
            # Decodificar la salida generada de IDs a texto legible
            pred_text = tokenizer.decode(output[0], skip_special_tokens=True)
            
            predictions.append(pred_text)
            references.append(example["quechua"]) # Añadir la referencia (traducción real)
            
    # Liberar memoria de GPU después de la evaluación de este modelo
    del model
    del tokenizer
    if device == "cuda":
        torch.cuda.empty_cache() # Limpiar la caché de la GPU

    return calculate_metrics(predictions, references) # Retornar las métricas calculadas

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import math
import numpy as np
import re

# Asegúrate de que esta variable esté definida antes de las clases
MAX_SEQ_LEN = 64 # O el valor que hayas usado en tu entrenamiento original, que parece ser 64 o 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Definición de la clase PositionalEmbedding
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq_len = MAX_SEQ_LEN):
        super().__init__()
        self.pos_embed_matrix = torch.zeros(max_seq_len, d_model, device=device)
        token_pos = torch.arange(0, max_seq_len, dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                             * (-math.log(10000.0)/d_model))
        self.pos_embed_matrix[:, 0::2] = torch.sin(token_pos * div_term)
        self.pos_embed_matrix[:, 1::2] = torch.cos(token_pos * div_term)
        self.pos_embed_matrix = self.pos_embed_matrix.unsqueeze(0).transpose(0,1)

    def forward(self, x):
        return x + self.pos_embed_matrix[:x.size(0), :]

# Definición de la clase MultiHeadAttention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model = 512, num_heads = 8):
        super().__init__()
        assert d_model % num_heads == 0, 'Embedding size not compatible with num heads'

        self.d_v = d_model // num_heads
        self.d_k = self.d_v
        self.num_heads = num_heads

        self.W_q = nn.Linear(d_model, d_model) ## query
        self.W_k = nn.Linear(d_model, d_model) ## Key
        self.W_v = nn.Linear(d_model, d_model) ## value
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, Q, K, V, mask = None):
        batch_size = Q.size(0)
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2 )
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2 )
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2 )

        weighted_values, attention = self.scale_dot_product(Q, K, V, mask)
        weighted_values = weighted_values.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads*self.d_k)
        weighted_values = self.W_o(weighted_values)

        return weighted_values, attention


    def scale_dot_product(self, Q, K, V, mask = None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention = F.softmax(scores, dim = -1)
        weighted_values = torch.matmul(attention, V)

        return weighted_values, attention


# Definición de la clase PositionFeedForward
class PositionFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))

# Definición de la clase EncoderSubLayer
class EncoderSubLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout = 0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.droupout1 = nn.Dropout(dropout)
        self.droupout2 = nn.Dropout(dropout)

    def forward(self, x, mask = None):
        attention_score, _ = self.self_attn(x, x, x, mask)
        x = x + self.droupout1(attention_score)
        x = self.norm1(x)
        x = x + self.droupout2(self.ffn(x))
        return self.norm2(x)

# Definición de la clase Encoder
class Encoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([EncoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

# Definición de la clase DecoderSubLayer
class DecoderSubLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, encoder_output, target_mask=None, encoder_mask=None):
        attention_score, _ = self.self_attn(x, x, x, target_mask)
        x = x + self.dropout1(attention_score)
        x = self.norm1(x)

        encoder_attn, _ = self.cross_attn(x, encoder_output, encoder_output, encoder_mask)
        x = x + self.dropout2(encoder_attn)
        x = self.norm2(x)

        ff_output = self.feed_forward(x)
        x = x + self.dropout3(ff_output)
        return self.norm3(x)

# Definición de la clase Decoder
class Decoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([DecoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, encoder_output, target_mask, encoder_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, target_mask, encoder_mask)
        return self.norm(x)

# Definición de la clase Transformer
class Transformer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers,
                 input_vocab_size, target_vocab_size,
                 max_len=MAX_SEQ_LEN, dropout=0.1):
        super().__init__()
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        self.pos_embedding = PositionalEmbedding(d_model, max_len)
        self.encoder = Encoder(d_model, num_heads, d_ff, num_layers, dropout)
        self.decoder = Decoder(d_model, num_heads, d_ff, num_layers, dropout)
        self.output_layer = nn.Linear(d_model, target_vocab_size)

    def forward(self, source, target):
        # Encoder mask
        source_mask, target_mask = self.mask(source, target)
        # Embedding and positional Encoding
        source = self.encoder_embedding(source) * math.sqrt(self.encoder_embedding.embedding_dim)
        source = self.pos_embedding(source)
        # Encoder
        encoder_output = self.encoder(source, source_mask)

        # Decoder embedding and postional encoding
        target = self.decoder_embedding(target) * math.sqrt(self.decoder_embedding.embedding_dim)
        target = self.pos_embedding(target)
        # Decoder
        output = self.decoder(target, encoder_output, target_mask, source_mask)

        return self.output_layer(output)


    def mask(self, source, target):
        source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
        target_mask = (target != 0).unsqueeze(1).unsqueeze(2)
        # The subsequent mask (look-ahead mask)
        size = target.size(1) # get seq_len for subsequent mask
        no_peak_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
        no_peak_mask = torch.autograd.Variable(torch.from_numpy(no_peak_mask) == 0).to(device)
        target_mask = target_mask & no_peak_mask
        return source_mask, target_mask

In [25]:
# Listas para almacenar las puntuaciones BLEU y ROUGE-L de cada ejecución para ambos modelos
bleu_scores_v1, rouge_scores_v1 = [], []
bleu_scores_v2, rouge_scores_v2 = [], []

# Bucle para ejecutar la evaluación múltiples veces (NUM_RUNS)
for run in range(NUM_RUNS):
    print(f"\n--- Ejecución {run+1}/{NUM_RUNS} ---")
    
    # Seleccionar un subconjunto aleatorio del dataset para esta ejecución
    # raw_dataset.shuffle(seed=run): Baraja el dataset de forma reproducible para cada ejecución
    # .select(range(NUM_SENTENCES)): Selecciona solo las primeras NUM_SENTENCES
    small_dataset = raw_dataset.shuffle(seed=run).select(range(NUM_SENTENCES))
    
    # --- Evaluar modelo v1 (modelo baseline) ---
    print("Evaluando modelo baseline...")
    # Llamar a la función evaluate_model con la ruta del modelo v1, el subconjunto de datos
    # y el nombre del tokenizador (que se cargará del Hub)
    metrics_v1 = evaluate_model(MODEL_PATH_V1, small_dataset, TOKENIZER_NAME_OR_PATH)
    bleu_scores_v1.append(metrics_v1["bleu"])
    rouge_scores_v1.append(metrics_v1["rouge"])
    
    # --- Evaluar modelo v2 (modelo refinado) ---
    print("Evaluando modelo refinado...")
    metrics_v2 = evaluate_model(MODEL_PATH_V2, small_dataset, TOKENIZER_NAME_OR_PATH)
    bleu_scores_v2.append(metrics_v2["bleu"])
    rouge_scores_v2.append(metrics_v2["rouge"])
    
    # Mostrar resultados parciales para la ejecución actual
    print(f"\nResultados ejecución {run+1}:")
    print(f"BLEU - v1: {metrics_v1['bleu']:.4f} | v2: {metrics_v2['bleu']:.4f}")
    print(f"ROUGE-L - v1: {metrics_v1['rouge']:.4f} | v2: {metrics_v2['rouge']:.4f}")

# (Opcional) Mostrar resultados finales promediados después de todas las ejecuciones
print("\n--- Resultados finales promedio ---")
print(f"BLEU promedio - v1: {np.mean(bleu_scores_v1):.4f} | v2: {np.mean(bleu_scores_v2):.4f}")
print(f"ROUGE-L promedio - v1: {np.mean(rouge_scores_v1):.4f} | v2: {np.mean(rouge_scores_v2):.4f}")


--- Ejecución 1/5 ---
Evaluando modelo baseline...


TypeError: Expected state_dict to be dict-like, got <class '__main__.Transformer'>.

In [ ]:
# Celda 5: Cálculo de estadísticas
# Medias
mean_bleu_v1 = np.mean(bleu_scores_v1)
mean_bleu_v2 = np.mean(bleu_scores_v2)
mean_rouge_v1 = np.mean(rouge_scores_v1)
mean_rouge_v2 = np.mean(rouge_scores_v2)

# Desviaciones estándar
std_bleu_v1 = np.std(bleu_scores_v1)
std_bleu_v2 = np.std(bleu_scores_v2)
std_rouge_v1 = np.std(rouge_scores_v1)
std_rouge_v2 = np.std(rouge_scores_v2)

# Diferencias
delta_bleu = mean_bleu_v2 - mean_bleu_v1
delta_rouge = mean_rouge_v2 - mean_rouge_v1

In [ ]:
# Celda 6: Visualización de resultados
# Tabla de resultados por fold
table_html = """
<h3>Resultados Comparativos por Fold</h3>
<table style="border-collapse: collapse; width: 100%;">
<tr style="background-color: #f2f2f2;">
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Fold</th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">BLEU Baseline</th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">BLEU Refinado</th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Δ BLEU</th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">ROUGE-L Baseline</th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">ROUGE-L Refinado</th>
<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Δ ROUGE-L</th>
</tr>
"""

for run in range(NUM_RUNS):
    delta_b = bleu_scores_v2[run]-bleu_scores_v1[run]
    delta_r = rouge_scores_v2[run]-rouge_scores_v1[run]
    
    table_html += f"""
<tr>
<td style="border: 1px solid #ddd; padding: 8px;">{run+1}</td>
<td style="border: 1px solid #ddd; padding: 8px;">{bleu_scores_v1[run]:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px;">{bleu_scores_v2[run]:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px; color: {'green' if delta_b >=0 else 'red'}">{delta_b:+.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px;">{rouge_scores_v1[run]:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px;">{rouge_scores_v2[run]:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px; color: {'green' if delta_r >=0 else 'red'}">{delta_r:+.4f}</td>
</tr>
"""

# Estadísticas finales
table_html += f"""
<tr style="font-weight: bold; background-color: #e6f3ff;">
<td style="border: 1px solid #ddd; padding: 8px;">Media ±STD</td>
<td style="border: 1px solid #ddd; padding: 8px;">{mean_bleu_v1:.4f} ±{std_bleu_v1:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px;">{mean_bleu_v2:.4f} ±{std_bleu_v2:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px; color: {'green' if delta_bleu >=0 else 'red'}">{delta_bleu:+.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px;">{mean_rouge_v1:.4f} ±{std_rouge_v1:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px;">{mean_rouge_v2:.4f} ±{std_rouge_v2:.4f}</td>
<td style="border: 1px solid #ddd; padding: 8px; color: {'green' if delta_rouge >=0 else 'red'}">{delta_rouge:+.4f}</td>
</tr>
</table>
"""

display(HTML(table_html))

In [ ]:
# Celda 7: Ejemplos de traducción
print("\n=== Ejemplos de Traducción (última ejecución) ===")

# Cargar modelos para ejemplos
tokenizer, model_v1 = load_local_model(MODEL_PATH_V1, TOKENIZER_PATH)
_, model_v2 = load_local_model(MODEL_PATH_V2, TOKENIZER_PATH)

for i in range(min(3, NUM_SENTENCES)):
    example = small_dataset[i]
    
    # Generar traducciones
    inputs = tokenizer(example["spanish"], return_tensors="pt", 
                     max_length=128, truncation=True, padding="max_length").to(device)
    
    # Modelo v1
    output_v1 = model_v1.generate(**inputs, max_length=128)
    pred_v1 = tokenizer.decode(output_v1[0], skip_special_tokens=True)
    
    # Modelo v2
    output_v2 = model_v2.generate(**inputs, max_length=128)
    pred_v2 = tokenizer.decode(output_v2[0], skip_special_tokens=True)
    
    # Mostrar resultados
    print(f"\nEjemplo {i+1}:")
    print(f"Español: {example['spanish']}")
    print(f"Referencia (Quechua): {example['quechua']}")
    print(f"Baseline: {pred_v1}")
    print(f"Refinado: {pred_v2}")